- Train a simple baeline with Transormers & Datasets

In [1]:
from datasets import load_dataset,DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer

In [2]:
import sys
sys.path.insert(1, '/tf/local/baneshi/PersianQA/')

In [9]:
# model_checkpoint = "HooshvareLab/bert-fa-base-uncased"
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 256 # The authorized overlap between two part of the context when splitting it is needed.
batch_size = 8
lr = 3e-6
epoch = 5

In [4]:
datasets = load_dataset("SajjadAyoubi/persian_qa")
datasets['train'][0]

  0%|          | 0/2 [00:00<?, ?it/s]

{'id': 1,
 'title': 'شرکت فولاد مبارکه اصفهان',
 'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم\u200cآبی زاینده\u200cرود شناخته می\u200cشود.',
 'questio

In [25]:
# datasets = load_dataset("SajjadAyoubi/persian_qa")

from src.read_ds import read_qa
train_dataset = read_qa("../dataset/pqa_train.json")
test_dataset = read_qa("../dataset/pqa_test.json")
dataset = {}
dataset['train'] = train_dataset
dataset['test'] = test_dataset
dataset = DatasetDict(dataset)
dataset['train'][0]


{'title': 'شرکت فولاد مبارکه اصفهان',
 'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم\u200cآبی زاینده\u200cرود شناخته می\u200cشود.',
 'question': 'شرکت 

In [5]:
model_checkpoint = "../models/roberta-fa-zwnj-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,)
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [10]:
# the datasets library does cashing itself, batched is multitreading for fast-tokenizer
tokenized_ds = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [11]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at ../models/roberta-fa-zwnj-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at ../models/roberta-fa-zwnj-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probab

In [12]:
args = TrainingArguments(
    f"result",
    evaluation_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.0001) 

In [13]:
ss = "/"+model_checkpoint.split("/")[-1]+"__PQA"
ss

'/roberta-fa-zwnj-base__PQA'

In [13]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['validation'],
    tokenizer=tokenizer)

In [ ]:
trainer.train()

In [11]:
# start training
trainer.train()

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9046
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5655


Epoch,Training Loss,Validation Loss
1,2.830800,2.338017
2,1.963400,2.205271
3,1.162500,2.368777
4,0.696300,2.780608
5,0.388500,3.265135


Saving model checkpoint to result/checkpoint-500
Configuration saved in result/checkpoint-500/config.json
Model weights saved in result/checkpoint-500/pytorch_model.bin
tokenizer config file saved in result/checkpoint-500/tokenizer_config.json
Special tokens file saved in result/checkpoint-500/special_tokens_map.json
Saving model checkpoint to result/checkpoint-1000
Configuration saved in result/checkpoint-1000/config.json
Model weights saved in result/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in result/checkpoint-1000/tokenizer_config.json
Special tokens file saved in result/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 930
  Batch size = 8
Saving model checkpoint to result/checkpoint-1500
Configuration saved in result/checkpoint-1500/config.json
Model weights saved in result/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in result/checkpoint-1500/tokenizer_config.json
Special tokens file saved in result/chec

TrainOutput(global_step=5655, training_loss=1.4617075676416318, metrics={'train_runtime': 2002.446, 'train_samples_per_second': 22.587, 'train_steps_per_second': 2.824, 'total_flos': 7897284154782288.0, 'train_loss': 1.4617075676416318, 'epoch': 5.0})

fine tuning on other dataset!!!

In [1]:
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 256 # The authorized overlap between two part of the context when splitting it is needed.
batch_size = 8
lr = 3e-5
epoch = 5

In [3]:
import sys
sys.path.insert(1, '/tf/local/baneshi/PersianQA/')

In [11]:
# datasets = load_dataset("SajjadAyoubi/persian_qa")

from src.read_ds import read_qa
train_dataset = read_qa("../dataset/Persian_SQuAD_train-v2.0.json")
test_dataset = read_qa("../dataset/Persian_SQuAD_dev-v2.0.json")
dataset = {}
dataset['train'] = train_dataset
dataset['test'] = test_dataset
dataset['train'][0]

{'title': 'بیانسه',
 'context': 'بیانسه جیزل نوولز-کارتر (/ biːˈjɒnseɪ / bee-YON-say) (متولد 4 سپتامبر 1981) خواننده ، ترانه سرا ، تهیه کننده ضبط و بازیگر آمریکایی است. وی در هوستون تگزاس متولد و بزرگ شد و در کودکی در مسابقات مختلف آواز و رقص شرکت کرد و در اواخر دهه 1990 به عنوان خواننده اصلی گروه دخترانه R&B Destiny\'s Child به شهرت رسید. با مدیریت پدرش ماتیو نولز ، این گروه به یکی از پرفروش ترین گروه های دختران جهان در تمام دوران تبدیل شد. وقفه آنها باعث انتشار اولین آلبوم بیانسه ، Dangerously in Love (2003) شد ، که او را به عنوان یک هنرمند انفرادی در سراسر جهان تثبیت کرد ، پنج جایزه گرمی را به دست آورد و تک آهنگ های شماره یک بی بیلبورد Hot "Crazy in Love" و "Baby Boy" را به نمایش گذاشت. .',
 'question': 'از چه زمانی بیانسه شروع به محبوبیت کرد؟',
 'id': '56be85543aeaaa14008c9063',
 'answers': {'answer_start': [221], 'text': ['در اواخر دهه 1990']}}

In [5]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,)
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [7]:
# the datasets library does cashing itself, batched is multitreading for fast-tokenizer
tokenized_ds = dataset.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

AttributeError: 'dict' object has no attribute 'map'

In [ ]:
model_name = "../notebooks/result/bert-fa-base-uncased__PQA/checkpoint-5000"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
args = TrainingArguments(
    f"result",
    evaluation_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.0001) 

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['validation'],
    tokenizer=tokenizer)

In [ ]:
# start training
trainer.train()